In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
import numpy as np
import os

from Software.GenuVP import runGNVP as gnvp

from PlaneDefinition.plane import Airplane as Plane
from PlaneDefinition.wing import Wing as wg
import PlaneDefinition.wing as wing

from Flight_Dynamics.disturbances import disturbance as disturb

from Database.getresults import Database_2D
from Database import DB3D , BASEGNVP

In [ ]:
HOMEDIR = os.getcwd()

# Airfoil Data

In [ ]:
db = Database_2D(HOMEDIR)
airfoils = db.getAirfoils()
polars2D = db.Data

# Get Plane

In [ ]:
Origin = np.array([0., 0., 0.])

In [ ]:
wingPos = np.array([0.0,0.0,0.0])
wingOrientation = np.array([2.8,0.0,0.0])

mainWing = wg(name="wing",
              airfoil=airfoils['NACA4415'],
              Origin=Origin + wingPos,
              Orientation=wingOrientation,           
              isSymmetric=True,
              span=2 * 1.130,
              sweepOffset=0,
              dihAngle=0,
              chordFun=wing.linearChord,
              chord=[0.159, 0.072],
              spanFun=wing.linSpan,
              N = 20,
              M = 5,
              mass = 0.670)
# mainWing.plotWing()

In [ ]:
elevatorPos = np.array([0.54,0.,0.])
elevatorOrientantion = np.array([0.,0.,0.])

elevator = wg(name = "tail",
              airfoil=airfoils['NACA0008'],
              Origin=Origin+ elevatorPos,
              Orientation=elevatorOrientantion,
              isSymmetric=True,
              span= 2 * 0.169,
              sweepOffset=0,
              dihAngle=0,
              chordFun=wing.linearChord,
              chord=[0.130, 0.03],
              spanFun=wing.linSpan,
              N = 15,
              M = 5,
              mass = 0.06)
# elevator.plotWing()

In [ ]:
rudderPos = np.array([0.47, 0., 0.01])
rudderOrientantion = np.array([0.0,0.0,90.0])

rudder = wg(name= "rudder",
            airfoil=airfoils['NACA0008'],
            Origin=Origin + rudderPos,
            Orientation= rudderOrientantion,
            isSymmetric=False,
            span= 0.160,
            sweepOffset=0.1,
            dihAngle=0,
            chordFun=wing.linearChord,
            chord=[0.2, 0.1],
            spanFun=wing.linSpan,
            N = 15,
            M = 5,
            mass = 0.04)
# rudder.plotWing()

In [ ]:
liftingSurfaces = [mainWing, elevator, rudder]
# liftingSurfaces = [rudder]
addedMasses = [
    (0.500 , np.array([-0.40, 0.0, 0.0])), # Motor
    (1.000 , np.array([0.090, 0.0, 0.0])), # Battery
    (0.900 , np.array([0.130, 0.0, 0.0])), # Payload
    ]
ap = Plane("Hermes", liftingSurfaces)
ap.accessDB(HOMEDIR, DB3D)
ap.visAirplane()
ap.addMasses(addedMasses)

In [ ]:
cleaning = False
calcGenu = True
calcBatchGenu = True
petrubationAnalysis = True
sensitivityAnalysis = True

## AoA Run

In [ ]:
AoAmax = 10
AoAmin = -6
NoAoA = (AoAmax - AoAmin) + 1
angles = np.linspace(AoAmin, AoAmax, NoAoA)

Uinf = 20
maxiter = 50
timestep = 5e-2

In [ ]:
ap.CASEDIR

In [ ]:
if calcBatchGenu == True:
    genuBatchArgs = [ap, BASEGNVP, polars2D,"Xfoil",
                     maxiter,timestep,Uinf, angles]
    ap.runSolver(gnvp.runGNVPangles,genuBatchArgs)
genuPolarArgs = [ap.CASEDIR, HOMEDIR]
ap.makePolars(gnvp.makePolar,genuPolarArgs)
ap.defineSim(Uinf, 1.225)
ap.save()


# Dynamics

In [ ]:
from Flight_Dynamics.dyn_plane import dyn_plane as dp

### Define and Trim Plane

In [ ]:
dyn = dp(ap,polars2D)

### Pertrubations

In [ ]:
dyn.allPerturb("Central")
dyn.disturbances = [dyn.disturbances[-1]]
dyn.get_pertrub()

In [ ]:
if petrubationAnalysis == True:
    genuBatchArgs = [dyn, BASEGNVP, polars2D,"Xfoil",
                     maxiter, timestep,
                     dyn.trim['U'], dyn.trim['AoA']]
    dyn.accessDB(HOMEDIR)
    dyn.runAnalysis(gnvp.runGNVPpertr,genuBatchArgs)
genuLogArgs = [dyn.DynDir, HOMEDIR]
dyn.logResults(gnvp.logResults,genuLogArgs)
dyn.save()

In [ ]:
if sensitivityAnalysis == True:
    for var in ['u','w','q','theta','v','p','r','phi']:
        space = np.logspace(np.log10(0.00001), np.log10(1), 10, base=10)
        space =[*-space,*space]

        dyn.sensitivityAnalysis(var, space)
        genuBatchArgs = [dyn, var, BASEGNVP, polars2D, "Xfoil",
                        maxiter, timestep,
                        dyn.trim['U'], dyn.trim['AoA']]
        # dyn.runAnalysis(gnvp.runGNVPsensitivity, genuBatchArgs)
        dyn.sensResults[var] = gnvp.logResults(
            f"{dyn.CASEDIR}/Sensitivity_{var}", HOMEDIR)
    dyn.save()

In [ ]:
dyn.trim

In [ ]:
dyn.scheme = "Backward"
dyn.longitudalStability('2D')
dyn.lateralStability('Potential')

In [ ]:
    #   Longitudinal derivatives
    #   Xu=    -0.16379         Cxu=   -0.049477
    #   Xw=      1.1494         Cxa=     0.34723
    #   Zu=     -3.0117         Czu=  -0.0035415
    #   Zw=     -19.528         CLa=      5.8991
    #   Zq=     -1.8321         CLq=      9.1645
    #   Mu= -0.00060463         Cmu=  -0.0015122
    #   Mw=    -0.45664         Cma=     -1.1421
    #   Mq=    -0.39098         Cmq=     -16.193

    #   Lateral derivatives
    #   Yv=     -0.5496         CYb=    -0.16602
    #   Yp=   -0.017022         CYp=  -0.0045504
    #   Yr=     0.32383         CYr=    0.086568
    #   Lv=    0.039731         Clb=   0.0053106
    #   Lp=     -5.6227         Clp=     -0.6651
    #   Lr=      1.0377         Clr=     0.12275
    #   Nv=     0.28879         Cnb=    0.038602
    #   Np=    -0.57586         Cnp=   -0.068117
    #   Nr=     -0.1259         Cnr=   -0.014893

In [ ]:
print(dyn.Along)
# [xu, xw, xq, xth]
# [zu, zw, zq, zth]
# [mu, mw, mq, mth]
# [0,  1 , 0 ,  0 ]

# Eigenvalue:  -4.524 + -8.045i | -4.524 + 8.045i           | -0.01025 + -0.602i        |-0.01025 + 0.602i
#   _____________________________________________________________________________________________________
# Eigenvector:           1 + 0i | 1 + 0i                    | 1 + 0i                   | 1 + 0i
         #      25.53 + -19.47i | 25.53 + 19.47i            | -0.03335 + 0.004629i     | -0.03335+-0.004629i
      #     -5.653+   -11.77i   |      -5.653+    11.77i    |      0.0371+ 0.002409i   |      0.0371+-0.002409i
      #      1.412+  0.09123i   |       1.412+ -0.09123i    |   -0.005049+  0.06154i   |   -0.005049+ -0.06154i
# dyn.AstarLong

print("\n\nEIGENVALUES:")
np.linalg.eig(dyn.Along)

eigvalLong, eigvecLong = np.linalg.eig(dyn.Along)
print(eigvalLong)


In [ ]:
print(dyn.Alat)
# [yv, yp , yr, yphi]
# [lv, lp , lr, lphi]
# [nv, n_p, nr, nph ]
# [0 , 1  , 0 , 0   ]

# XFLR5
# -0.173375         -0.00536967              -20.63                9.81
#  0.243269             -30.274             5.56351                   0
#  0.902122            -2.37023            -0.28454                   0
#         0                   1                   0                   0

# dyn.AstarLat

print("\n\nEIGENVALUES:")
eigvalLat, eigvecLat =  np.linalg.eig(dyn.Alat)
print(eigvalLat)

In [ ]:
# plot the complex numbers
import matplotlib.pyplot as plt 

# extract real part
x = [ele.real for ele in eigvalLong]
# extract imaginary part
y=[ele.imag for ele in eigvalLong]
plt.scatter(x, y, color = "r")

# extract real part
x = [ele.real for ele in eigvalLat]
# extract imaginary part
y = [ele.imag for ele in eigvalLat]
plt.scatter(x, y, color="b",marker = 'x')

plt.ylabel('Imaginary')
plt.xlabel('Real')
plt.grid()
plt.show()

In [ ]:
liftingSurfaces = [mainWing, elevator, rudder]
# liftingSurfaces = [rudder]
addedMasses = [
    (0.500, np.array([-0.40, 0.0, 0.0])),  # Motor
    (1.000, np.array([0.090, 0.0, 0.0])),  # Battery
    (0.900, np.array([0.130, 0.0, 0.0])),  # Payload
]
ap = Plane("Hermes2", liftingSurfaces)
ap.accessDB(HOMEDIR, DB3D)
ap.visAirplane()
ap.addMasses(addedMasses)

AoAmax = 10
AoAmin = -6
NoAoA = (AoAmax - AoAmin) + 1
angles = np.linspace(AoAmin, AoAmax, NoAoA)

Uinf = 20
maxiter = 70
timestep = 1e-1


if calcBatchGenu == True:
    genuBatchArgs = [ap, BASEGNVP, polars2D, "Xfoil",
                     maxiter, timestep, Uinf, angles]
    ap.runSolver(gnvp.runGNVPangles, genuBatchArgs)
genuPolarArgs = [ap.CASEDIR, HOMEDIR]
ap.makePolars(gnvp.makePolar, genuPolarArgs)
ap.defineSim(Uinf, 1.225)
ap.save()
